# Project Scenario
You have been hired as a data engineer by research organization. Your boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

Your job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

# Import Libraries

In [72]:
import requests
import pandas as pd
import numpy as np
import sqlite3
from bs4 import BeautifulSoup
from datetime import datetime

# Initialize the entities

In [77]:
Exchange_rate = pd.read_csv('exchange_rate.csv')
Output_csv = 'Largest_banks_data.csv'
db_name = 'Banks.db'
log_file = 'Largest_banks_data.txt'
table_name = 'Largest_banks_data'
table_attr=['Name',' MC_USD_Billion']

In [65]:
URL = "https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks"

# Extraction

In [6]:
def extract(URL,table_attr):
    df =  pd.DataFrame(columns=table_attr)
    html_page = requests.get(URL).text
    soup = BeautifulSoup(html_page,'html.parser')
    rows = soup.find('tbody').find_all('tr')
    #print(rows[0])
    for row in rows:
        col = row.find_all('td')
        if len(col)>=3:
            data_dict = {'Name':col[1].get_text(strip=True),' MC_USD_Billion': col[2].get_text(strip=True)}
            df1 = pd.DataFrame(data_dict,index=[0])
            df=pd.concat([df,df1],ignore_index=[0])
    return df

In [15]:
#df = extract(URL,table_attr)

# Tansformation
The data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file

In [14]:
Exchange_rate

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


In [17]:
#df[' MC_USD_Billion'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10 entries, 0 to 9
Series name:  MC_USD_Billion
Non-Null Count  Dtype 
--------------  ----- 
10 non-null     object
dtypes: object(1)
memory usage: 212.0+ bytes


In [61]:
def transformation(df):
    df[' MC_USD_Billion'] = df[' MC_USD_Billion'].astype(float)
    for i in range(len(Exchange_rate)):
        df[Exchange_rate.Currency[i]] = np.round(df[' MC_USD_Billion']* Exchange_rate.Rate[i],2)
    return df

In [62]:
#print(transformation(df))


                                      Name   MC_USD_Billion     EUR     GBP  \
0                           JPMorgan Chase           432.92  402.62  346.34   
1                          Bank of America           231.52  215.31  185.22   
2  Industrial and Commercial Bank of China           194.56  180.94  155.65   
3               Agricultural Bank of China           160.68  149.43  128.54   
4                                HDFC Bank           157.91  146.86  126.33   
5                              Wells Fargo           155.87  144.96  124.70   
6                        HSBC Holdings PLC           148.90  138.48  119.12   
7                           Morgan Stanley           140.83  130.97  112.66   
8                  China Construction Bank           139.82  130.03  111.86   
9                            Bank of China           136.81  127.23  109.45   

        INR  
0  35910.71  
1  19204.58  
2  16138.75  
3  13328.41  
4  13098.63  
5  12929.42  
6  12351.26  
7  11681.85  
8  1

# Loading
The processed information table is to be saved locally in a CSV format and as a database table.

In [66]:
def load_csv(df,Output_csv):
    df.to_csv(Output_csv)

def load_db(df,table_name,sql_connection):
    df.to_sql(table_name,sql_connection,if_exists='replace',index=False)

In [70]:
def query_table(query_statement,sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

# Logging
Progress the whole process into a log file

In [74]:
def log_progress(message):
    time_format = '%Y-%h-%d-%H :%M :%S'
    now = datetime.now()
    timestamp = now.strftime(time_format)
    with open (log_file,'a') as f:
        f.write (timestamp + ' '+ message)

# Calling the functions

In [80]:
log_progress("Preliminaries Complete. Initiating ETL Process\n")
df = extract(URL,table_attr)
log_progress("Data Extraction Complete. Initiating Transformation Process\n")
df = transformation(df)
log_progress("Data Transformation Complete. Initiating Loading Process\n")
load_csv(df,Output_csv)
log_progress("Data Loaded to a CSV file\n")
conn = sqlite3.connect(db_name)
log_progress("SQL Connection Initialized\n")
load_db(df,table_name,conn)
log_progress("Data Loaded to a Database Table. Running the query\n")
query_statement = f"SELECT * from {table_name} "
query_table(query_statement, conn)
log_progress('Process Complete.\n')
conn.close()

SELECT * from Largest_banks_data 
                                      Name   MC_USD_Billion     EUR     GBP  \
0                           JPMorgan Chase           432.92  402.62  346.34   
1                          Bank of America           231.52  215.31  185.22   
2  Industrial and Commercial Bank of China           194.56  180.94  155.65   
3               Agricultural Bank of China           160.68  149.43  128.54   
4                                HDFC Bank           157.91  146.86  126.33   
5                              Wells Fargo           155.87  144.96  124.70   
6                        HSBC Holdings PLC           148.90  138.48  119.12   
7                           Morgan Stanley           140.83  130.97  112.66   
8                  China Construction Bank           139.82  130.03  111.86   
9                            Bank of China           136.81  127.23  109.45   

        INR  
0  35910.71  
1  19204.58  
2  16138.75  
3  13328.41  
4  13098.63  
5  12929.42 